Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# Logistic Regression
batch_size = 128
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    # L2 Regularization
    
    regularization = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    
    loss = loss + regularization_beta * regularization

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 22.192091
Minibatch accuracy: 7.8%
Validation accuracy: 9.9%
Minibatch loss at step 100: 4.150168
Minibatch accuracy: 70.3%
Validation accuracy: 70.7%
Minibatch loss at step 200: 3.419696
Minibatch accuracy: 77.3%
Validation accuracy: 74.6%
Minibatch loss at step 300: 3.049146
Minibatch accuracy: 74.2%
Validation accuracy: 74.8%
Minibatch loss at step 400: 2.276532
Minibatch accuracy: 75.8%
Validation accuracy: 75.4%
Minibatch loss at step 500: 2.195283
Minibatch accuracy: 74.2%
Validation accuracy: 75.2%
Minibatch loss at step 600: 2.177200
Minibatch accuracy: 74.2%
Validation accuracy: 76.2%
Minibatch loss at step 700: 2.712954
Minibatch accuracy: 72.7%
Validation accuracy: 77.0%
Minibatch loss at step 800: 2.010926
Minibatch accuracy: 75.8%
Validation accuracy: 76.1%
Minibatch loss at step 900: 1.958153
Minibatch accuracy: 71.9%
Validation accuracy: 77.4%
Minibatch loss at

In [8]:
# https://github.com/giubil/udacity-deep-learning/blob/master/3_regularization.ipynb
# Neural Networks

batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(hidden, weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
        tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 523.023132
Minibatch accuracy: 10.2%
Validation accuracy: 29.7%
Minibatch loss at step 100: 192.123657
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 200: 166.364334
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 300: 151.421265
Minibatch accuracy: 82.8%
Validation accuracy: 78.5%
Minibatch loss at step 400: 131.261353
Minibatch accuracy: 86.7%
Validation accuracy: 81.9%
Minibatch loss at step 500: 125.247787
Minibatch accuracy: 89.8%
Validation accuracy: 82.0%
Minibatch loss at step 600: 117.272758
Minibatch accuracy: 91.4%
Validation accuracy: 80.8%
Minibatch loss at step 700: 112.168007
Minibatch accuracy: 85.9%
Validation accuracy: 81.3%
Minibatch loss at step 800: 111.283134
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 900: 100.345222
Minibatch accuracy: 89.1%
Validation accuracy: 82

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
# Neural Networks

batch_size = 32
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(hidden, weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
        tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)
    

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 489.458984
Minibatch accuracy: 15.6%
Validation accuracy: 13.6%
Minibatch loss at step 100: 199.549133
Minibatch accuracy: 59.4%
Validation accuracy: 71.7%
Minibatch loss at step 200: 196.435120
Minibatch accuracy: 84.4%
Validation accuracy: 75.2%
Minibatch loss at step 300: 161.740143
Minibatch accuracy: 75.0%
Validation accuracy: 73.7%
Minibatch loss at step 400: 182.350739
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 500: 149.179413
Minibatch accuracy: 81.2%
Validation accuracy: 76.3%
Minibatch loss at step 600: 120.192886
Minibatch accuracy: 96.9%
Validation accuracy: 76.1%
Minibatch loss at step 700: 118.203003
Minibatch accuracy: 90.6%
Validation accuracy: 76.5%
Minibatch loss at step 800: 117.328835
Minibatch accuracy: 65.6%
Validation accuracy: 74.9%
Minibatch loss at step 900: 109.719772
Minibatch accuracy: 81.2%
Validation accuracy: 75

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [17]:
# Neural Networks

batch_size = 32
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(hidden, weights_output) + biases_output

    #logits = tf.matmul(tf.nn.dropout(hidden, 0.5), weights_output) + biases_output

    #logits = tf.nn.dropout(tf.matmul(hidden, weights_output), 0.5) * 2.0 + biases_output

    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    #loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
        tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)
    

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 285.407013
Minibatch accuracy: 21.9%
Validation accuracy: 22.6%
Minibatch loss at step 100: 83.706657
Minibatch accuracy: 56.2%
Validation accuracy: 72.1%
Minibatch loss at step 200: 40.803543
Minibatch accuracy: 81.2%
Validation accuracy: 74.6%
Minibatch loss at step 300: 11.470514
Minibatch accuracy: 81.2%
Validation accuracy: 74.9%
Minibatch loss at step 400: 65.616341
Minibatch accuracy: 71.9%
Validation accuracy: 73.9%
Minibatch loss at step 500: 15.972925
Minibatch accuracy: 84.4%
Validation accuracy: 73.0%
Minibatch loss at step 600: 10.178455
Minibatch accuracy: 84.4%
Validation accuracy: 76.9%
Minibatch loss at step 700: 10.815131
Minibatch accuracy: 84.4%
Validation accuracy: 76.8%
Minibatch loss at step 800: 13.233374
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 900: 20.032024
Minibatch accuracy: 75.0%
Validation accuracy: 74.0%
Minib

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
